In [1]:
import numpy as np

In [4]:
N = 700
K = 4
np.random.seed(0)
prob = np.random.rand(4)
prob = prob / prob.sum()
n_samples = [int(N*p) for p in prob]
n_samples[-1] += N - sum(n_samples)

X = []
for i in range(K):
    shift = np.random.randint(1, 20, (2,))
    affine = np.random.rand(2, 2) * 2 - 1
    X.append(np.dot(np.random.randn(n_samples[i], 2), affine) + shift)
X = np.vstack(X)

In [6]:
import torch

In [10]:
data = torch.from_numpy(X).float()

In [11]:
D = 2
gamma = torch.rand(N, K)
gamma = gamma / gamma.sum(dim=1, keepdim=True)
# data: (N, D)
# gamma: (N, K)
# alpha : (1, K)
# means : (K, D)
# covs: (K, D, D)

In [12]:
def estimate_gaussian_parameters(data, gamma):
    N = len(data)
    alpha = gamma.sum(dim=0) / N
    means = torch.mm(gamma.T, data) / gamma.sum(dim=0).unsqueeze(1)
    diffs = data.unsqueeze(1) - means.unsqueeze(0)  # (N, K, D)
    covs = torch.einsum('nkdj,nkjt->nkdt', diffs.unsqueeze(3), diffs.unsqueeze(2))  # (N, K, D, D)
    covs = (covs * gamma[:, :, None, None]).sum(dim=0) / gamma.sum(dim=0)[:, None, None]
    return alpha, means, covs

In [13]:
def estimate_log_gamma(data, alpha, means, covs):
    weighted_log_prob = torch.log(alpha) + estimate_log_gaussian_prob(data, means, covs)
    norm = torch.logsumexp(weighted_log_prob, dim=1)
    log_gamma = weighted_log_prob - norm.unsqueeze(1)
    return log_gamma

In [14]:
def estimate_log_gaussian_prob(data, means, covs):
    N, D = data.shape
    covs_chol = torch.linalg.cholesky(covs)
    precision_col = torch.inverse(covs_chol).permute(0, 2, 1)
    log_det = torch.logdet(precision_col)
    diffs = data[:, None, :] - means[None, :, :]
    log_prob = torch.einsum('nkd,kdj->nkj', diffs,  precision_col)
    log_prob = (log_prob ** 2).sum(dim=2)
    return -0.5 * (D * torch.log(2 * torch.Tensor([torch.pi])) + log_prob) + log_det

In [15]:
TOL = 1e-3
alpha, means, covs = estimate_gaussian_parameters(data, gamma)
for i in range(500):
    log_gamma = estimate_log_gamma(data, alpha, means, covs)
    gamma = torch.exp(log_gamma)
    _alpha, _means, _covs = estimate_gaussian_parameters(data, gamma)
    if torch.allclose(alpha, _alpha, TOL) and torch.allclose(means, _means, TOL) and torch.allclose(covs, _covs, TOL):
        print('finish @ iter %d' % i)
        break
    alpha, means, covs = _alpha, _means, _covs

finish @ iter 21


In [16]:
def log_likelihood(data, alpha, means, covs):
    weighted_log_prob = torch.log(alpha) + estimate_log_gaussian_prob(data, means, covs)
    return torch.logsumexp(weighted_log_prob, dim=1)

In [17]:
x = torch.linspace(-2, 15, 100)
y = torch.linspace(2.5, 25, 100)
x, y = torch.meshgrid(x, y)

/Users/wangyulong/anaconda3/envs/py3.7/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [18]:
coords = torch.stack([x.ravel(), y.ravel()]).T

In [19]:
scores = log_likelihood(coords, alpha, means, covs)

In [21]:
assign_idx = estimate_log_gamma(data, alpha, means, covs).argmax(dim=1)